# NLP -Beginner 任务1 基于机器学习的文本分类
实现基于logistic/softmax regression的文本分类
1. 参考
   1. [文本分类](文本分类.md)
   2. 《[神经网络与深度学习](https://nndl.github.io/)》 第2/3章
2. 数据集：[Classify the sentiment of sentences from the Rotten Tomatoes dataset](https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews)
3. 实现要求：NumPy
4. 需要了解的知识点：

   1. 文本特征表示：Bag-of-Word，N-gram
   2. 分类器：logistic/softmax  regression，损失函数、（随机）梯度下降、特征选择
   3. 数据集：训练集/验证集/测试集的划分
5. 实验：
   1. 分析不同的特征、损失函数、学习率对最终分类性能的影响
   2. shuffle 、batch、mini-batch 
6. 时间：两周

### 初始化 
导入相应数据，模块包

In [62]:
import numpy as np
import csv
import random
import pandas as pd
import math

with open('train.tsv') as f:
    tsvreader = csv.reader(f, delimiter='\t')
    temp = list(tsvreader)



# 初始化
raw_data = temp[1:]  # 忽略了表头
raw_data = np.array(raw_data)[:,2:]
max_item = 1000      # 提取前max_item个数据，否则数据量太大 BoWs就有150000*16000的矩阵
raw_data = raw_data[:max_item,:]
raw_data.shape

(1000, 2)

### DataLoader类
目标：实现DataLoader类，将数据集根据Bag of Words 模型或者 N-gram 模型， 转化为特征向量和标签的形式,以X和y返回

In [63]:
class DataLoader:
    def __init__(self, raw_data, method = 'BoWs'):
        self.raw_data = raw_data
        self.method = method
        self.words_dict = dict()
        self.len = 0
        self.X = None
        self.y = self.raw_data[:,-1]    # 这是标签
        self.N = raw_data.shape[0]    # 数据集的sentence条目
    
    def __call__(self):
        self.get_words()
        if self.method == 'BoWs':
            self.BoWs()
        elif self.method == "N_grams":
            self.N_grams()
        else:
            raise Exception('Wrong method, please use BoWs or N_grams')
        return self.X,self.y
        
        
    def get_words(self):
        '''
        统计所有出现的词数量,构建词典
        '''
        for item in self.raw_data:
            s = item[0]
            s = s.upper()   # 大小写统一是合理并且必要的
            s = s.split()
            for word in s:
                if word not in self.words_dict:
                    self.words_dict[word] = len(self.words_dict)    # 漂亮的一步（参考）：单词在词典中的值与自然数集对应，从而变为其在特征矩阵中的下表
        self.len = len(self.words_dict)
        self.X = np.zeros((self.N,self.len))    # 初始化特征矩阵
    
    def BoWs(self):
        for i in range(self.N):
            s = self.raw_data[i][0]
            s = s.upper()
            s = s.split()
            for word in s:
                self.X[i][self.words_dict[word]] += 1
            
    def N_grams():
        pass


# 测试一下
dataloader = DataLoader(raw_data,method='BoWs')
X, y = dataloader()
print(X.shape)
print(y.shape)

(1000, 363)
(1000,)


In [73]:
from random import shuffle


def train_valid_split(X, y, train_percent=0.8):
    '''
    输入处理好的数据，随机打乱，返回 X_train, X_valid, y_train, y_valid
    '''
    n = len(X)
    indices = list(range(n))
    random.shuffle(indices)
    X = X[indices]
    y = y[indices]
    train_num = int(n*train_percent)
    X_train = X[:train_num,:]
    X_valid = X[train_num:]
    y_train = y[:train_num]
    y_valid = y[train_num:]
    return X_train, X_valid, y_train, y_valid

# 测试一下
X_train, X_valid, y_train, y_valid = train_valid_split(X,y)
print(X_train.shape,X_valid.shape,y_train.shape,y_valid.shape)

(800, 363) (200, 363) (800,) (200,)
